In [1]:
import numpy as np
import pandas as pd
import seaborn

In [2]:
# Pandas is rather sophisitacted in its treatment of indexes. For instance, it lets you use indices with multiple levels,
# that is, it let's you use compound labels. I will show you how to create such a MultiIndex object, how to use it to index 
# DataFrames, and how to apply the stack and unstack Pandas function, which trade between levels of an index and column names.

In [3]:
# The dataset flights contains data about passengers who took flights between 1949 and 1960.
flights = seaborn.load_dataset("flights")

In [4]:
# With the head method of a Pandas DataFrame, I'm only showing you the very beginning of the frame.
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [39]:
# In this case, it's convenient to use a Pandas MultiIndex by telling Pandas to index the dataframe flights with both, year 
# and month.
flights_indexed = flights.set_index(['year','month'])
flights_indexed.head()

passengers
year month               
1949 January          112
     February         118
     March            132
     April            129
     May              121

In [6]:
# The IPython notebook shows us both year and month in bold type to point out that together they comprise the MultiIndex.

In [7]:
# Now, we can use many different forms of indexing.
# For instance, we may just select a specific year, using loc since we're using the value of an index.
# This selects rows placed only on the first level of the MultiIndex.
flights_indexed.loc[1949]

,passengers
month,
January,112
February,118
March,132
April,129
May,121
June,135
July,148
August,148
September,136


In [8]:
# We could also select a range of years using a slice, say between 1949 and 1950.
# Since the slicing is on an explicit index, the end year, 1950, is included.
flights_indexed.loc[1949:1950]

passengers
year month                
1949 January           112
     February          118
     March             132
     April             129
     May               121
     June              135
     July              148
     August            148
     September         136
     October           119
     November          104
     December          118
1950 January           115
     February          126
     March             141
     April             135
     May               125
     June              149
     July              170
     August            170
     September         158
     October           133
     November          114
     December          140

In [9]:
# We could also select a specific year and month,
flights_indexed.loc[1949,'January']

passengers    112
Name: (1949, January), dtype: int64

In [10]:
# or a range of months.
# For this, first we go and select the entire year, 1949, and then use slicing.
flights_indexed.loc[1949].loc['January':'June']

,passengers
month,
January,112
February,118
March,132
April,129
May,121
June,135


In [11]:
# The unstack method let's us trade between the levels of a MultiIndex and column names.
flights_unstacked = flights_indexed.unstack()

In [12]:
# As you can see, the second level of the MultiIndex has been used to create columns.
flights_unstacked

passengers                                                               \
month    January February March April  May June July August September October   
year                                                                            
1949         112      118   132   129  121  135  148    148       136     119   
1950         115      126   141   135  125  149  170    170       158     133   
1951         145      150   178   163  172  178  199    199       184     162   
1952         171      180   193   181  183  218  230    242       209     191   
1953         196      196   236   235  229  243  264    272       237     211   
1954         204      188   235   227  234  264  302    293       259     229   
1955         242      233   267   269  270  315  364    347       312     274   
1956         284      277   317   313  318  374  413    405       355     306   
1957         315      301   356   348  355  422  465    467       404     347   
1958         340      318   362   348  363  435  491    505       404     359   
1959         360      342   406   396  420  472  548    559       463     407   
1960         417      391   419   461  472  535  622    606       508     461   

                         
month November December  
year                     
1949       104      118  
1950       114      140  
1951       146      166  
1952       172      194  
1953       180      201  
1954       203      229  
1955       237      278  
1956       271      306  
1957       305      336  
1958       310      337  
1959       362      405  
1960       390      432

In [13]:
# We can now perform a sum along the rows. This gives us totals for each year
flights_unstacked.sum(axis=1)

year
1949    1520
1950    1676
1951    2042
1952    2364
1953    2700
1954    2867
1955    3408
1956    3939
1957    4421
1958    4572
1959    5140
1960    5714
dtype: int64

In [14]:
#note, the loaded dataset has some issues when modifying the column structure. So we do it this way
flights = pd.read_csv('flights.csv')
flights_unstacked = flights.set_index(['year','month']).unstack()
total = flights_unstacked.sum(axis=1) #this line is important
#flights_unstacked['passengers','total'] = flights_unstacked.sum(axis=1) will give incorrect results!!!!

In [18]:
# and we can assign the result to a new column.
# Now the names of the columns have also become compound labels, so we need to specify two levels: passengers and total.
flights_unstacked['passengers','total'] = total
flights_unstacked

passengers                                                        \
month      April August December February January July June March  May   
year                                                                     
1949         129    148      118      118     112  148  135   132  121   
1950         135    170      140      126     115  170  149   141  125   
1951         163    199      166      150     145  199  178   178  172   
1952         181    242      194      180     171  230  218   193  183   
1953         235    272      201      196     196  264  243   236  229   
1954         227    293      229      188     204  302  264   235  234   
1955         269    347      278      233     242  364  315   267  270   
1956         313    405      306      277     284  413  374   317  318   
1957         348    467      336      301     315  465  422   356  355   
1958         348    505      337      318     340  491  435   362  363   
1959         396    559      405      342     360  548  472   406  420   
1960         461    606      432      391     417  622  535   419  472   

                                        
month November October September total  
year                                    
1949       104     119       136  1520  
1950       114     133       158  1676  
1951       146     162       184  2042  
1952       172     191       209  2364  
1953       180     211       237  2700  
1954       203     229       259  2867  
1955       237     274       312  3408  
1956       271     306       355  3939  
1957       305     347       404  4421  
1958       310     359       404  4572  
1959       362     407       463  5140  
1960       390     461       508  5714

In [19]:
# The new column total has appeared at the end. We can then go back to a stacked MultiIndex.
# We'll call this flights_restacked and just call the method stack on our dataframe.
flights_restacked = flights_unstacked.stack()
# Just as we expected, now the second level of the MultiIndex includes also total.
flights_restacked

passengers
year month                
1949 April             129
     August            148
     December          118
     February          118
     January           112
     July              148
     June              135
     March             132
     May               121
     November          104
     October           119
     September         136
     total            1520
1950 April             135
     August            170
     December          140
     February          126
     January           115
     July              170
     June              149
     March             141
     May               125
     November          114
     October           133
     September         158
     total            1676
1951 April             163
     August            199
     December          166
     February          150
...                    ...
1958 November          310
     October           359
     September         404
     total            4572
1959 April             396
     August            559
     December          405
     February          342
     January           360
     July              548
     June              472
     March             406
     May               420
     November          362
     October           407
     September         463
     total            5140
1960 April             461
     August            606
     December          432
     February          391
     January           417
     July              622
     June              535
     March             419
     May               472
     November          390
     October           461
     September         508
     total            5714

[156 rows x 1 columns]

In [36]:
flights_restacked.loc[1949,'total']['passengers']

1520

In [42]:
# The indexing syntax can be a little complicated because Python doesn't allow some things, such as putting slices inside 
# tuples. In this case, you can use the Pandas index slice object.
# For instance, say that we want to see only the total number of passengers for each year.
# For that, we can use the loc indexing objects, and then you use a Pandas index slice object to take care of the rows.
# For instance, take all the years but only select total for the second level of the MultiIndex.
# Then we need to add a standard selection for the columns.
flights_restacked.loc[pd.IndexSlice[:,'total'],'passengers']

year  month
1949  total    1520
1950  total    1676
1951  total    2042
1952  total    2364
1953  total    2700
1954  total    2867
1955  total    3408
1956  total    3939
1957  total    4421
1958  total    4572
1959  total    5140
1960  total    5714
Name: passengers, dtype: int64

In [21]:
# Of course, it's also possible to use advanced indexing.
# For instance, by selecting all the rows where the numbers of passengers is larger than a certain value,
# and then using this boolean mask to index flights_restacked.
# This returns the subset of all the rows.
flights_restacked[flights_restacked['passengers'] > 120]

passengers
year month                
1949 April             129
     August            148
     July              148
     June              135
     March             132
     May               121
     September         136
     total            1520
1950 April             135
     August            170
     December          140
     February          126
     July              170
     June              149
     March             141
     May               125
     October           133
     September         158
     total            1676
1951 April             163
     August            199
     December          166
     February          150
     January           145
     July              199
     June              178
     March             178
     May               172
     November          146
     October           162
...                    ...
1958 November          310
     October           359
     September         404
     total            4572
1959 April             396
     August            559
     December          405
     February          342
     January           360
     July              548
     June              472
     March             406
     May               420
     November          362
     October           407
     September         463
     total            5140
1960 April             461
     August            606
     December          432
     February          391
     January           417
     July              622
     June              535
     March             419
     May               472
     November          390
     October           461
     September         508
     total            5714

[149 rows x 1 columns]